In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import dill as pickle
import pandas as pd
import aesara.tensor as aet
import pycmtensor as cmt
from pycmtensor import aet as aet
from pycmtensor.expressions import Beta, Weights
from pycmtensor.optimizers import Adam
from pycmtensor.results import Results
from pycmtensor.models import MNLogit

In [3]:
class ResLogitLayer:
    def __init__(self, u, w_in, w_out, activation=None):
        
        assert w_in.shape[1].eval() == w_out.shape[0].eval()
        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        
        if isinstance(u, (list, tuple)):
            assert len(u) == w_in.shape[0].eval(), f"index.0 of w_in must be of the same length as u"
            self.U = aet.stacklists(u).flatten(2)
        else:
            self.U = u

        self.w_in = w_in()
        self.w_out = w_out()
        if activation == None:
            activation = aet.sigmoid
            
        h = activation(aet.dot(self.U.T, self.w_in))
        output = activation(aet.dot(h, self.w_out)).T
        self.params = [self.w_in, self.w_out]
        self.output = output + self.U

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"

In [4]:
nb_path = os.path.abspath("")
df_rp = pd.read_csv("data/model_average_RP.csv")
df_rp.columns = df_rp.columns.str.upper()

df_rp["PURPOSE_WORK"] = 0
df_rp["PURPOSE_HOME"] = 0
df_rp["PURPOSE_LEISURE"] = 0
df_rp.loc[df_rp["PURPOSE"].str.split(" ", expand=True)[0] == "Work", "PURPOSE_WORK"] = 1
df_rp.loc[df_rp["PURPOSE"].str.split(" ", expand=True)[0] == "Home", "PURPOSE_HOME"] = 1
df_rp.loc[df_rp["PURPOSE"].isin([
	"Cinema or other night out", 
	"Clothes shopping", 
	"Family Activity", 
	"Leisure Other", 
	"Museum/cultural", 
	"Social", 
	"Restaurant", 
	"Sports activity", 
	"Shopping - Major", 
	]), "PURPOSE_LEISURE"] = 1

df_rp["WEEKEND"] = 0
df_rp.loc[df_rp["DAY_OF_THE_WEEK_START"].isin(["Saturday", "Sunday"]), "WEEKEND"] = 1
df_rp.drop(["PURPOSE", "DAY_OF_THE_WEEK_START"], axis=1, inplace=True)
df_rp.fillna(0, inplace=True)

db = cmt.Database("model_average_rp", df_rp, choiceVar="CHOICE")
globals().update(db.variables)

# additional steps to format database
db.data['CHOICE'] -= 1
for i in [
	"TOTAL_CAR_COST", "BUS_COST_TOTAL_PER_LEG", "RAIL_COST_TOTAL_PER_LEG", "TAXI_COST"
]:
	db.data[i] /= 100.

for i in [
	"CAR_DISTANCE_KM", "BUS_DISTANCE_KM", "RAIL_DISTANCE_KM", "TAXI_DISTANCE_KM", 
	"CYCLING_DISTANCE_KM", "WALKING_DISTANCE_KM",
]:
	db.data[i] /= 100.

for i in [
	"CAR_TRAVEL_TIME_MIN", "BUS_TRAVEL_TIME_MIN", "RAIL_TRAVEL_TIME_MIN", 
	"TAXI_TRAVEL_TIME_MIN", "CYCLING_TRAVEL_TIME_MIN", "WALKING_TRAVEL_TIME_MIN",
	"BUS_IVT_TIME_MIN", "BUS_ACCESS_EGRESS_TIME_MIN", "RAIL_IVT_TIME_MIN", 
	"RAIL_ACCESS_EGRESS_TIME_MIN",
]:
	db.data[i] /= 60.

# db.autoscale(variables=c, verbose=False)

# specify Beta parameters
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
b_dist = Beta("b_dist", 0.0, None, None, 0)

b_IVT = Beta("b_IVT", 0.0, None, None, 0)
b_egress = Beta("b_egress", 0.0, None, None, 0)
b_transfers = Beta("b_transfers", 0.0, None, None, 0)

b_ncar = Beta("b_ncar", 0.0, None, None, 0)
b_nbicycle = Beta("b_nbicycle", 0.0, None, None, 0)

b_weekend_car = Beta("b_weekend_car", 0.0, None, None, 0)
b_weekend_pt = Beta("b_weekend_pt", 0.0, None, None, 0)
b_weekend_phys = Beta("b_weekend_phys", 0.0, None, None, 0)

b_cleeds_car = Beta("b_cleeds_car", 0.0, None, None, 0)
b_cleeds_pt = Beta("b_cleeds_pt", 0.0, None, None, 0)
b_cleeds_phys = Beta("b_cleeds_phys", 0.0, None, None, 0)

b_female_car = Beta("b_female_car", 0.0, None, None, 0)
b_female_pt = Beta("b_female_pt", 0.0, None, None, 0)
b_female_phys = Beta("b_female_phys", 0.0, None, None, 0)

b_age_car = Beta("b_age_car", 0.0, None, None, 0)
b_age_pt = Beta("b_age_pt", 0.0, None, None, 0)
b_age_phys = Beta("b_age_phys", 0.0, None, None, 0)

b_uk_car = Beta("b_uk_car", 0.0, None, None, 0)
b_uk_pt = Beta("b_uk_pt", 0.0, None, None, 0)
b_uk_phys = Beta("b_uk_phys", 0.0, None, None, 0)

b_edu_gba_car = Beta("b_edu_gba_car", 0.0, None, None, 0)
b_edu_gba_pt = Beta("b_edu_gba_pt", 0.0, None, None, 0)
b_edu_gba_phys = Beta("b_edu_gba_phys", 0.0, None, None, 0)

b_n_emp_car = Beta("b_n_emp_car", 0.0, None, None, 0)
b_n_emp_pt = Beta("b_n_emp_pt", 0.0, None, None, 0)
b_n_emp_phys = Beta("b_n_emp_phys", 0.0, None, None, 0)

b_mar_car = Beta("b_mar_car", 0.0, None, None, 0)
b_mar_pt = Beta("b_mar_pt", 0.0, None, None, 0)
b_mar_phys = Beta("b_mar_phys", 0.0, None, None, 0)

b_hhsize_car = Beta("b_hhsize_car", 0.0, None, None, 0)
b_hhsize_pt = Beta("b_hhsize_pt", 0.0, None, None, 0)
b_hhsize_phys = Beta("b_hhsize_phys", 0.0, None, None, 0)

b_ppinc_lt30k_car = Beta("b_ppinc_lt30k_car", 0.0, None, None, 0)
b_ppinc_lt30k_pt = Beta("b_ppinc_lt30k_pt", 0.0, None, None, 0)
b_ppinc_lt30k_phys = Beta("b_ppinc_lt30k_phys", 0.0, None, None, 0)

b_hhinc_lt50k_car = Beta("b_hhinc_lt50k_car", 0.0, None, None, 0)
b_hhinc_lt50k_pt = Beta("b_hhinc_lt50k_pt", 0.0, None, None, 0)
b_hhinc_lt50k_phys = Beta("b_hhinc_lt50k_phys", 0.0, None, None, 0)

b_ft_car = Beta("b_ft_car", 0.0, None, None, 0)
b_ft_pt = Beta("b_ft_pt", 0.0, None, None, 0)
b_ft_phys = Beta("b_ft_phys", 0.0, None, None, 0)

asc_car = Beta("asc_car", 0.0, None, None, 1)
asc_bus = Beta("asc_bus", 0.0, None, None, 0)
asc_rail = Beta("asc_rail", 0.0, None, None, 0)
asc_taxi = Beta("asc_taxi", 0.0, None, None, 0)
asc_cycling = Beta("asc_cycling", 0.0, None, None, 0)
asc_walking = Beta("asc_walking", 0.0, None, None, 0)

# 1: car, 2: bus, 3: rail, 4: taxi, 5: cycling, 6: walking.
# specify weight parameters
W1 = Weights("ResNet_01a", (6, 16), 0, True)
W2 = Weights("ResNet_01b", (16, 6), 0, True)

# W3 = Weights("ResNet_02a", (6, 16), 0, True)
# W4 = Weights("ResNet_02b", (16, 6), 0, True)

U_1 = (
	b_cost * db["TOTAL_CAR_COST"] + b_time * db["CAR_TRAVEL_TIME_MIN"]
	+ b_dist * db["CAR_DISTANCE_KM"]
	+ b_ncar * db["N_CAR"] 
	+ b_weekend_car * db["WEEKEND"] 
	+ b_cleeds_car * db["CITY.LEEDS"] 
	+ b_female_car * db["FEMALE"]
	+ b_age_car * (db["AGE"] <= 3)
	+ b_uk_car * (db["BRITISH_ORIGIN"])
	# + b_edu_gba_car * ((db["MASTERS"] + db["PHD"]) > 0)
	+ b_n_emp_car * db["NB_HH_EMPLOYED"]
	# + b_mar_car * aet.eq(db["MAR"], 2)
	+ b_hhsize_car * db["HH_SIZE"]
	# + b_hhinc_lt50k_car * (db["INCOME_HH"] <= 5)
	# + b_ppinc_lt30k_car * (db["INCOME_PERSO"] <= 3)
	+ b_ft_car * db["OCC_FULL_TIME"]
	+ asc_car
)

U_2 = (
	b_cost * db["BUS_COST_TOTAL_PER_LEG"] + b_time * db["BUS_TRAVEL_TIME_MIN"]
	+ b_dist * db["BUS_DISTANCE_KM"]
	+ b_IVT * db["BUS_IVT_TIME_MIN"] + b_egress * db["BUS_ACCESS_EGRESS_TIME_MIN"]
	+ b_transfers * db["BUS_TRANSFERS"]
	+ b_weekend_pt * db["WEEKEND"] 
	+ b_cleeds_pt * db["CITY.LEEDS"]
	# + b_female_pt * db["FEMALE"]
	+ b_age_pt * (db["AGE"] <= 3)
	+ b_uk_pt * (db["BRITISH_ORIGIN"])
	+ b_edu_gba_pt * ((db["MASTERS"] + db["PHD"]) > 0)
	+ b_n_emp_pt * db["NB_HH_EMPLOYED"]
	+ b_mar_pt * aet.eq(db["MAR"], 2)
	+ b_hhsize_pt * db["HH_SIZE"]
	+ b_hhinc_lt50k_pt * (db["INCOME_HH"] <= 5)
	# + b_ppinc_lt30k_pt * (db["INCOME_PERSO"] <= 3)
	+ b_ft_pt * db["OCC_FULL_TIME"]
	+ asc_bus 
)

U_3 = (
	b_cost * db["RAIL_COST_TOTAL_PER_LEG"] + b_time * db["CAR_TRAVEL_TIME_MIN"]
	+ b_dist * db["RAIL_DISTANCE_KM"]
	+ b_IVT * db["RAIL_IVT_TIME_MIN"] + b_egress * db["RAIL_ACCESS_EGRESS_TIME_MIN"]
	+ b_transfers * db["RAIL_TRANSFERS"]
	+ b_weekend_pt * db["WEEKEND"] 
	+ b_cleeds_pt * db["CITY.LEEDS"]
	# + b_female_pt * db["FEMALE"]
	+ b_age_pt * (db["AGE"] <= 3)
	+ b_uk_pt * (db["BRITISH_ORIGIN"])
	+ b_edu_gba_pt * ((db["MASTERS"] + db["PHD"]) > 0)
	+ b_n_emp_pt * db["NB_HH_EMPLOYED"]
	+ b_mar_pt * aet.eq(db["MAR"], 2)
	+ b_hhsize_pt * db["HH_SIZE"]
	+ b_hhinc_lt50k_pt * (db["INCOME_HH"] <= 5)
	# + b_ppinc_lt30k_pt * (db["INCOME_PERSO"] <= 3)
	+ b_ft_pt * db["OCC_FULL_TIME"]
	+ asc_rail
)

U_4 = (
	b_cost * db["TAXI_COST"] + b_time * db["TAXI_TRAVEL_TIME_MIN"]
	+ b_dist * db["TAXI_DISTANCE_KM"]
	+ asc_taxi
)

U_5 = (
	b_time * db["CYCLING_TRAVEL_TIME_MIN"] + b_dist * db["CYCLING_DISTANCE_KM"]
	+ b_nbicycle * db["N_BICYCLE"] 
	+ b_weekend_phys * db["WEEKEND"]
	+ b_cleeds_phys * db["CITY.LEEDS"] 
	+ b_female_phys * db["FEMALE"]
	+ b_age_phys * (db["AGE"] <= 3)
	# + b_uk_phys * (db["BRITISH_ORIGIN"])
	+ b_edu_gba_phys * ((db["MASTERS"] + db["PHD"]) > 0)
	+ b_n_emp_phys * db["NB_HH_EMPLOYED"]
	+ b_mar_phys * aet.eq(db["MAR"], 2)
	# + b_hhsize_phys * db["HH_SIZE"]
	+ b_hhinc_lt50k_phys * (db["INCOME_HH"] <= 5)
	+ b_ppinc_lt30k_phys * (db["INCOME_PERSO"] <= 3)
	+ b_ft_phys * db["OCC_FULL_TIME"]
	+ asc_cycling
)

U_6 = (
	b_time * db["WALKING_TRAVEL_TIME_MIN"] + b_dist * db["WALKING_DISTANCE_KM"]
	+ b_weekend_phys * db["WEEKEND"]
	+ b_cleeds_phys * db["CITY.LEEDS"] 
	+ b_female_phys * db["FEMALE"]
	+ b_age_phys * (db["AGE"] <= 3)
	# + b_uk_phys * (db["BRITISH_ORIGIN"])
	+ b_edu_gba_phys * ((db["MASTERS"] + db["PHD"]) > 0)
	+ b_n_emp_phys * db["NB_HH_EMPLOYED"]
	+ b_mar_phys * aet.eq(db["MAR"], 2)
	# + b_hhsize_phys * db["HH_SIZE"]
	+ b_hhinc_lt50k_phys * (db["INCOME_HH"] <= 5)
	+ b_ppinc_lt30k_phys * (db["INCOME_PERSO"] <= 3)
	+ b_ft_phys * db["OCC_FULL_TIME"]
	+ asc_walking
)

In [5]:
# Associate utility functions with the list
U = [U_1, U_2, U_3, U_4, U_5, U_6]
U = ResLogitLayer([U_1, U_2, U_3, U_4, U_5, U_6], W1, W2).output
# U = ResLogitLayer(U, W3, W4).output

# Associate the availability conditions with the alternatives
av = [
    db["AVAIL_CAR"], 
    db["AVAIL_BUS"],
    db["AVAIL_RAIL"],
    db["AVAIL_TAXI"],
    db["AVAIL_CYCLING"],
    db["AVAIL_WALKING"],
]

# rll = ResLogitLayer(U, W1, W2)
model = MNLogit(U, av, database=db, name="mymodel")
model.add_params(locals())


# train function
model = cmt.train(model, database=db, optimizer=Adam, batch_size=256, lr_init=0.04, max_epoch=2500, notebook=True)

with open("mymodel.pkl", "rb") as f:
    model = pickle.load(f)

Building model...
dataset: model_average_rp (10120)
batch size: 256
batches per epoch: 39
validation frequency: 39

Training model...


Loglikelihood:  -11238.696849  Score: 0.757

Epoch    0/97500:   0%|          | 0.00/97.5k [00:00<?, ?it/s]

Optimization complete with accuracy of 87.787% with maximum loglikelihood reached @ epoch 2253.



In [6]:
result = Results(model, db, show_weights=True)


Results for model: mymodel
Build time: 00:00:18
Estimation time: 00:14:15
Estimation rate: 2.925 epochs/s
Seed value: 999
Number of Beta parameters: 41
Tensor size: 192
Sample size: 10120
Excluded data: 0
Init loglikelihood: -11238.697
Final loglikelihood: -3586.957
Final loglikelihood reached at: epoch 2253
Likelihood ratio test: 15303.480
Accuracy: 87.787%
Rho square: 0.681
Rho bar square: 0.660
Akaike Information Criterion: 7639.91
Bayesian Information Criterion: 9322.70
Final gradient norm: 0.038



In [7]:
result.print_beta_statistics()
result.print_correlation_matrix()
result.print_nn_weights()

C:\Users\Melvin\miniconda3\envs\icmc\lib\site-packages\pandas\core\series.py:2442: RuntimeWarning: overflow encountered in multiply
  result = self._values.round(decimals)


Statistical Analysis:
                        Value   Std err     t-test   p-value Rob. Std err Rob. t-test Rob. p-value
asc_bus             -0.032092  0.168268  -0.190722  0.848743      0.25525    -0.12573     0.899946
asc_cycling         -2.311938  0.042662 -54.191674       0.0     0.239972   -9.634207          0.0
asc_rail             1.289741  0.182552   7.065066       0.0     0.207324    6.220897          0.0
asc_taxi            -2.264862  0.054023 -41.923822       0.0     0.005095  -444.52575          0.0
asc_walking           1.05901  0.077441  13.675091       0.0     0.238023    4.449187     0.000009
b_IVT                2.207453  0.033112  66.666909       0.0     0.218929   10.082961          0.0
b_age_car           -1.944019  0.041373 -46.987471       0.0      0.14115  -13.772728          0.0
b_age_phys          -0.731334  0.084266  -8.678848       0.0     0.074683   -9.792521          0.0
b_age_pt            -1.727965  0.064812 -26.661129       0.0     0.265334   -6.512406  